In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [21]:
# We load the data
# There are three dataframes to load: dim_pa, dim_claims, dim_date. The bridge is a dictrionary to merge the data
dim_pa=pd.read_csv("dim_pa.csv")
dim_claims=pd.read_csv("dim_claims.csv")
dim_date=pd.read_csv("dim_date.csv")
# We load the bridge
bridge=pd.read_csv("bridge.csv")

We start by reordering the data and merging the relevant dataframes we will consider for the clasification problem:

In [22]:
# We add a column dim_pa_id to the dataframe dim_claims:
dim_claims['dim_pa_id']=bridge['dim_pa_id']
# We add a column dim_date_id to the dataframe dim_claims:
dim_claims['dim_date_id']=bridge['dim_date_id']

In [23]:
# We now merge the dim_pa with dim_pa based on the column dim_pa_id
df = pd.merge(dim_pa, dim_claims, on="dim_pa_id", how="left")

In [24]:
# Next we merge the resulting dataframe with dim_date based on the column dim_date_id
df = pd.merge(df, dim_date, on="dim_date_id", how="left")

In [25]:
# Here we we clean rhe data: For our classification problem, we the following columns are irrelevant:
df=df.drop(['pharmacy_claim_approved','calendar_day','day_of_week','calendar_month','calendar_year','date_val'],axis=1)

In [26]:
# We extract the columns' names
df.columns[:-1]

Index(['dim_pa_id', 'correct_diagnosis', 'tried_and_failed',
       'contraindication', 'pa_approved', 'dim_claim_id', 'bin', 'drug',
       'reject_code', 'dim_date_id', 'is_weekday', 'is_workday'],
      dtype='object')

In [27]:
# Assign the following numbers:
# drug A --> 0
# drug B --> 1
# drug C --> 2
df['drug_type']=0
df.loc[df.drug == 'B','drug_type'] = 1
df.loc[df.drug == 'C','drug_type'] = 2

In [28]:
# Assign the following numbers:
# reject code 70 --> 0
# reject code 75  --> 1
# reject code 76  --> 2
df['rejected_code']=0
df.loc[df.reject_code == 75.0,'rejected_code'] = 1
df.loc[df.reject_code == 76.0,'rejected_code'] = 2

In [29]:
# Assign the following numbers:
# bin number 417740--> 1
# bin number 417380  --> 2
# bin number 417614  --> 3
#bin number 999001 --->4

df['bin_code']=1
df.loc[df.bin == 417380,'bin_code'] = 2
df.loc[df.bin == 417614,'bin_code'] = 3
df.loc[df.bin == 999001,'bin_code'] = 4

In [30]:
# We now clean up the dataframe and re-order the columns.
# drop the unwanted columns
df=df.drop(['drug','reject_code','bin','dim_pa_id','dim_claim_id','dim_date_id'],axis=1)


In [31]:
df

,correct_diagnosis,tried_and_failed,contraindication,pa_approved,is_weekday,is_workday,is_holiday,drug_type,rejected_code,bin_code
0,1,1,0,1,0,0,1,0,1,2
1,1,0,0,1,0,0,1,0,2,1
2,0,0,1,1,0,0,1,0,1,2
3,1,1,0,1,0,0,1,0,2,4
4,0,1,0,1,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...
555946,1,0,1,0,1,1,0,2,0,2
555947,1,0,1,1,1,1,0,2,1,1
555948,1,1,1,1,1,1,0,2,1,1
555949,1,0,0,1,1,1,0,2,1,1


In [36]:
# We extract the columns' names
df.columns[:-1]

Index(['rejected_code', 'drug_type', 'correct_diagnosis', 'tried_and_failed',
       'contraindication', 'is_holiday', 'is_weekday', 'is_workday'],
      dtype='object')

In [37]:
#re-order the columns
df=df[['rejected_code', 'drug_type','correct_diagnosis', 'tried_and_failed',
       'contraindication','is_holiday','is_weekday','is_workday','pa_approved',
    ]]

In [38]:
# the following dataframe will be the final datafram we will work with:
approval_df=df

In [40]:
approval_df

,rejected_code,drug_type,correct_diagnosis,tried_and_failed,contraindication,is_holiday,is_weekday,is_workday,pa_approved
0,1,0,1,1,0,1,0,0,1
1,2,0,1,0,0,1,0,0,1
2,1,0,0,0,1,1,0,0,1
3,2,0,1,1,0,1,0,0,1
4,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
555946,0,2,1,0,1,0,1,1,0
555947,1,2,1,0,1,0,1,1,1
555948,1,2,1,1,1,0,1,1,1
555949,1,2,1,0,0,0,1,1,1


In [41]:
#Save the dataframe as cover_my_med_df for later use.
df.to_csv('cover_my_med_df.csv')